# SCRIPT FOR UPOADING SEED DISTRIBUTION 2021

In [5]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1xkzCIEflGMhi8ymUBdQpZPZPfjT4ItPSzKeOsUNW2H4/export?format=csv'

r = requests.get(google_sheet_url)
data = r.content
df = pd.read_csv(BytesIO(data))
local_index = []
df = df.fillna(0)
df.head(4)

,Sl. No,Rl. No,Distribution/Return,Date,Agent Name,Agent Code,Village Name,Supervisor Name,Cluster Name,From 50,To 50,No of Packets 50,Acre 50,Total Acre
0,1,1,Distribution,25-02-2021,M Parameshwaran,AS055,Gundlupet,siddapa,Sargur,1.0,30.0,30,15.0,100.0
1,2,2,Distribution,25-02-2021,M Parameshwaran,AH034,Gundlupet,srinivasa,HD Kote,31.0,70.0,40,20.0,100.0
2,3,3,Distribution,25-02-2021,Nandisha,ASO65,Heerehalli,srinivasa,HD Kote,71.0,90.0,20,10.0,60.0
3,4,4,Distribution,25-02-2021,Puttamallappa,AH043,Teranimunti,srinivasa,HD Kote,91.0,110.0,20,10.0,75.0


In [2]:
InputGoods.objects.filter(code='IG21_S0004').delete()
InputGoods.objects.filter(code='IG21_S0005').delete()
InputGoods.objects.filter(code='IC21013').delete()
InputPacketInventory.objects.filter(packet_code='IC21008').delete()
InputPacketInventory.objects.filter(packet_code='IC21009').delete()
InputPacketInventory.objects.filter(packet_code='IC21013').delete()
InputPacketInventory.objects.filter(packet_code='IC21010').delete()
InputPacketInventory.objects.filter(packet_code='IC21011').delete()
ComboIssueRequest.objects.filter(input_combo_id__in=[8,9]).delete()

(0, {})

In [6]:
def generate_request_code(input_combo_id):
    code_bank_obj = ComboIssueRequestCodeBank.objects.filter(input_combo_id=8)[0]
    last_digit_code = code_bank_obj.last_digit
    new_code = last_digit_code + 1
    value = str(code_bank_obj.input_combo.name) + str(new_code).zfill(3)
    code_bank_obj.last_digit = new_code
    code_bank_obj.save()
    return value

def senior_supervisor_id(supervisor_id):
    superior_id = UserHierarchyMap.objects.get(subordinate=supervisor_id).superior_id
    return superior_id

def get_active_season_id():
    season_obj = Season.objects.get(is_active=True)
    return season_obj.id

In [8]:
@transaction.atomic
def register_agent_inventory(storage_list, combo_issue_request_id, dispatch_date, agri_officer):
    print(storage_list)
    print(combo_issue_request_id)
    print(dispatch_date)
    
    sid = transaction.savepoint()
    try:
        storage_list = storage_list
        for storage in storage_list:
            print('crossed')
            store_quantity = int(storage['entered_qty'])
            storage_obj = InputSubStoreInventory.objects.get(id=storage['id'])
            available_qty = storage_obj.quantity_now
            print('crossed')
            storage_obj.quantity_now = available_qty - int(storage['entered_qty'])
            print('crossed')
            storage_obj.quantity_now_time = datetime.datetime.now()

            storage_obj.save()
            combo_issue_request_obj = ComboIssueRequest.objects.get(id=combo_issue_request_id)
            agents = ComboIssueRequestAgentMap.objects.filter(combo_issue_request_id=combo_issue_request_id)
           
            for agent_combo in agents:
                if store_quantity == 0:
                    break
                if SubStoreIssueLabelAgentMap.objects.filter(agent_id=agent_combo.agent.id, combo_issue_request_agent_map_id=agent_combo.id).exists():
                    label_qty = SubStoreIssueLabelAgentMap.objects.filter(agent_id=agent_combo.agent.id, combo_issue_request_agent_map_id=agent_combo.id).count()
                else:
                    label_qty = 0
                if agent_combo.quantity_in_numbers == label_qty:
                    print('more')
                else:
                    required_qty = agent_combo.quantity_in_numbers - label_qty
                    if required_qty > store_quantity:
                        required_qty = store_quantity
                    if not AgentInventory.objects.filter(combo_issue_request_id=combo_issue_request_id, agent_id=agent_combo.agent.id).exists():
                        agent_inventory_obj = AgentInventory(
                            season_id=get_active_season_id(),
                            combo_issue_request_id=combo_issue_request_id,
                            agent_id=agent_combo.agent.id,
                            date_of_receipt=datetime.datetime.now(),
                            quantity_at_receipt=agent_combo.quantity_in_numbers,
                            quantity_now=agent_combo.quantity_in_numbers,
                            quantity_now_time=datetime.datetime.now(),
                            unit_id=4,
                            price_per_item=10,
                            created_by_id=agri_officer,
                            modified_by_id=agri_officer,
                        )
                        agent_inventory_obj.save()
                    else:
                        agent_inventory_obj = AgentInventory.objects.get(combo_issue_request_id=combo_issue_request_id, agent_id=agent_combo.agent.id)
                    label_from = InputSubStoreInventoryPacketLabel.objects.filter(input_sub_store_inventory_id=storage_obj.id, stock_status_id=1).order_by('id')[0].label
                    if not AgentInventoryStoreLabelRangeMap.objects.filter(agent_inventory_id=agent_inventory_obj.id, input_sub_store_inventory_id=storage_obj.id).exists():
                        agent_inventory_store_lable_map_obj = AgentInventoryStoreLabelRangeMap(agent_inventory_id=agent_inventory_obj.id,
                                                                                               input_sub_store_inventory_id=storage_obj.id,
                                                                                               label_range_from=label_from)
                        agent_inventory_store_lable_map_obj.save()
                    else:
                        agent_inventory_store_lable_map_obj = AgentInventoryStoreLabelRangeMap.objects.get(agent_inventory_id=agent_inventory_obj.id, input_sub_store_inventory_id=storage_obj.id)
                    for i in range(1, required_qty+1):
                        agent_invent_label_obj = InputSubStoreInventoryPacketLabel.objects.filter(input_sub_store_inventory_id=storage_obj.id, stock_status_id=1).order_by('id')[0]
                        agent_invent_label_obj.stock_status_id=2
                        agent_invent_label_obj.save()
                        new_table_obj = SubStoreIssueLabelAgentMap(input_sub_store_inventory_id=storage_obj.id,
                                                                agent_id=agent_combo.agent.id,
                                                                combo_issue_request_agent_map_id=agent_combo.id,
                                                                agent_inventory_id=agent_inventory_obj.id,
                                                                label=agent_invent_label_obj.label,
                                                                created_by_id=agri_officer,
                                                                modified_by_id=agri_officer)
                        new_table_obj.save()
                        label_to = agent_invent_label_obj.label
                    agent_inventory_store_lable_map_obj.label_range_to = label_to
                    agent_inventory_store_lable_map_obj.save()
                    store_quantity -= required_qty
                    print(agent_combo.agent.first_name , ' ', label_from , ' - ', label_to)
            
            if storage_obj.quantity_now == 0:
                storage_obj.label_range_start = '0'
                storage_obj.label_range_end = '0'
            else:
                label_from = InputSubStoreInventoryPacketLabel.objects.filter(input_sub_store_inventory_id=storage_obj.id, stock_status_id=1).order_by('id')[0].label
                storage_obj.label_range_start = label_from
            storage_obj.save()
            combo_issue_request_obj.max_status_id = 8
            combo_issue_request_obj.max_status_date = datetime.datetime.now()
            combo_issue_request_obj.dispatched_by_id = agri_officer
            combo_issue_request_obj.dispatch_status_id = 3
            combo_issue_request_obj.dispatch_date = dispatch_date
            combo_issue_request_obj.save()
#         transaction.savepoint_commit(sid)
        
        print('success')
        print('------------------------')
    except Exception as e:
        print('error on {}'.format(e))
        print('---------------------------problem')
        transaction.savepoint_rollback(sid)

### 100 grams upload

In [5]:
for index, row in df.iterrows():
    agent_code = row['Agent Code']
    print('run')
    if UserProfile.objects.filter(code = agent_code).exists():
       
        village_name = row['Village Name']
        date_string = row['Date']
        date = pd.to_datetime(date_string).date()

        supervisor_id = User.objects.get(username=row['Supervisor Name']).id
        superior_id = senior_supervisor_id(supervisor_id)
        input_combo_obj = InputCombo.objects.get(id=8)
        combo_request_obj = ComboIssueRequest(request_code=generate_request_code(8),
                                            season_id=3,
                                            input_combo_id=8,
                                            quantity_in_numbers=row['No of Packets 100'],
                                            quantity_for_area=row['Acre 100'],
                                            expected_date=date,
                                            issue_rised_date=date,
                                            max_status_id=6,
                                            max_status_date=date,
                                            supervisor_id=User.objects.get(username=row['Supervisor Name']).id,
                                            senior_supervisor_id=superior_id,
                                            senior_supervisor_status_id=1,
                                            senior_supervisor_status_date=date,
                                            assitant_manager_id=79,
                                            assitant_manager_status_id=1,
                                            assitant_manager_status_date=date,
                                            agri_officer_status_id = 1,
                                            agri_officer_status_date=date,
                                            gm_id = 78,
                                            gm_status_id =1,
                                            gm_status_date=date)

        if senior_supervisor_id == 80:
            combo_request_obj.agri_officer_id=82,
            agri_officer = 82
        else:
            combo_request_obj.agri_officer_id=99
            agri_officer = 99
        combo_request_obj.save()
 
        agent_id = UserProfile.objects.get(code = agent_code).user_id
        combo_request_agent_obj = ComboIssueRequestAgentMap(combo_issue_request_id=combo_request_obj.id,
                                                                    agent_id=agent_id,
                                                                    issue_rised_date=date,
                                                                    quantity_in_numbers=row['No of Packets 100'],
                                                                    max_status_id=2,
                                                                    shop_id=1)

        combo_request_agent_obj.save()
        main_df = pd.DataFrame(list(InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo__id__in=[8,9]).order_by('id').values()))
        master_input_dict = []
        new_val = row['No of Packets 100']
        
        for index, row_item in main_df.iterrows():
            print(row_item['id'],row_item['quantity_now'])
            if new_val < row_item['quantity_now'] or new_val == row_item['quantity_now']:
                temp_dict = {}
                temp_dict['id'] = row_item.id
                temp_dict['entered_qty'] = new_val
                master_input_dict.append(temp_dict)
                break;
            else:
                temp_dict = {}
                temp_dict['id'] = row_item.id
                temp_dict['entered_qty'] = row_item['quantity_now']
                master_input_dict.append(temp_dict)
                new_val = new_val - row_item['quantity_now']
        register_agent_inventory(master_input_dict, combo_request_obj.id, date, agri_officer)

print('insterted')   

run
43 138
[{'id': 43, 'entered_qty': 85}]
194
2021-02-25
crossed
crossed
crossed


/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.expected_date received a naive datetime (2021-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.issue_rised_date received a naive datetime (2021-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/kultivate/Softwares/vir

Parameswaran_sargur   S21F11M1  -  S21F185M1
success
------------------------
run
43 53
44 379
[{'id': 43, 'entered_qty': 53}, {'id': 44, 'entered_qty': 27}]
195
2021-02-25
crossed
crossed
crossed
Parameswaran_hd_kote   S21F186M1  -  S21F1138M1
crossed
crossed
crossed


/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:21:22.209086) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.dispatch_date received a naive datetime (2021-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:21:22.248224) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/k

Parameswaran_hd_kote   S21F1139M1  -  S21F1165M1
success
------------------------
run
43 0
44 352
[{'id': 43, 'entered_qty': 0}, {'id': 44, 'entered_qty': 50}]
196
2021-02-25
crossed
crossed
crossed
crossed
crossed
crossed
Nandisha   S21F1166M1  -  S21F1215M1
success
------------------------
run


...

/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:21:38.413799) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:21:38.417664) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:21:38.420399) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datet

Ravi M S   CCGB35611ST KIT  -  CCGB35951ST KIT
success
------------------------
run
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
[{'id': 43, 'entered_qty': 0}, {'id': 44, 'entered_qty': 0}, {'id': 45, 'entered_qty': 0}, {'id': 46, 'entered_qty': 0}, {'id': 47, 'entered_qty': 0}, {'id': 48, 'entered_qty': 0}, {'id': 49, 'entered_qty': 0}, {'id': 50, 'entered_qty': 0}, {'id': 51, 'entered_qty': 0}, {'id': 52, 'entered_qty': 0}]
281
2021-03-19
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
success
------------------------
run
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
[{'id': 43, 'entered_qty': 0}, {'id': 44, 'entered_qty': 0}, {'id': 45, 'entered_qty': 0}, {'id': 46, 'entered_qty': 0}, {'id': 47, 'entered_qty': 0}, {'id': 48, 'entered_qty': 0}, {'id': 49, 'entered_qty': 0}, {'id': 50, '

/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:21:38.719330) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:21:38.756540) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:21:38.762863) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s

crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
success
------------------------
run
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
[{'id': 43, 'entered_qty': 0}, {'id': 44, 'entered_qty': 0}, {'id': 45, 'entered_qty': 0}, {'id': 46, 'entered_qty': 0}, {'id': 47, 'entered_qty': 0}, {'id': 48, 'entered_qty': 0}, {'id': 49, 'entered_qty': 0}, {'id': 50, 'entered_qty': 0}, {'id': 51, 'entered_qty': 0}, {'id': 52, 'entered_qty': 0}]
284
2021-03-26
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
success
------------------------
run
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
[{'id': 43, 'entered_qty': 0}, {'id': 44, 'entered_qty': 0}, {'id': 45, 'enter

/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:21:38.920729) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:21:38.924193) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:21:38.926789) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datet

In [7]:
def generate_request_code(input_combo_id):
    code_bank_obj = ComboIssueRequestCodeBank.objects.filter(input_combo_id=9)[0]
    last_digit_code = code_bank_obj.last_digit
    new_code = last_digit_code + 1
    value = str(code_bank_obj.input_combo.name) + str(new_code).zfill(3)
    code_bank_obj.last_digit = new_code
    code_bank_obj.save()
    return value

### 50 grams upload

In [9]:
for index, row in df.iterrows():
    agent_code = row['Agent Code']
    print('run')
    if UserProfile.objects.filter(code = agent_code).exists():
       
        village_name = row['Village Name']
        date_string = row['Date']
        date = pd.to_datetime(date_string).date()

        supervisor_id = User.objects.get(username=row['Supervisor Name']).id
        superior_id = senior_supervisor_id(supervisor_id)
        input_combo_obj = InputCombo.objects.get(id=8)
        combo_request_obj = ComboIssueRequest(request_code=generate_request_code(8),
                                            season_id=3,
                                            input_combo_id=9,
                                            quantity_in_numbers=row['No of Packets 50'],
                                            quantity_for_area=row['Acre 50'],
                                            expected_date=date,
                                            issue_rised_date=date,
                                            max_status_id=6,
                                            max_status_date=date,
                                            supervisor_id=User.objects.get(username=row['Supervisor Name']).id,
                                            senior_supervisor_id=superior_id,
                                            senior_supervisor_status_id=1,
                                            senior_supervisor_status_date=date,
                                            assitant_manager_id=79,
                                            assitant_manager_status_id=1,
                                            assitant_manager_status_date=date,
                                            agri_officer_status_id = 1,
                                            agri_officer_status_date=date,
                                            gm_id = 78,
                                            gm_status_id =1,
                                            gm_status_date=date)

        if senior_supervisor_id == 80:
            combo_request_obj.agri_officer_id=82,
            agri_officer = 82
        else:
            combo_request_obj.agri_officer_id=99
            agri_officer = 99
        combo_request_obj.save()
 
        agent_id = UserProfile.objects.get(code = agent_code).user_id
        combo_request_agent_obj = ComboIssueRequestAgentMap(combo_issue_request_id=combo_request_obj.id,
                                                                    agent_id=agent_id,
                                                                    issue_rised_date=date,
                                                                    quantity_in_numbers=row['No of Packets 50'],
                                                                    max_status_id=2,
                                                                    shop_id=1)

        combo_request_agent_obj.save()
        main_df = pd.DataFrame(list(InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo__id__in=[9]).order_by('id').values()))
        master_input_dict = []
        new_val = row['No of Packets 50']
        
        for index, row_item in main_df.iterrows():
            print(row_item['id'],row_item['quantity_now'])
            if new_val < row_item['quantity_now'] or new_val == row_item['quantity_now']:
                temp_dict = {}
                temp_dict['id'] = row_item.id
                temp_dict['entered_qty'] = new_val
                master_input_dict.append(temp_dict)
                break;
            else:
                temp_dict = {}
                temp_dict['id'] = row_item.id
                temp_dict['entered_qty'] = row_item['quantity_now']
                master_input_dict.append(temp_dict)
                new_val = new_val - row_item['quantity_now']
        register_agent_inventory(master_input_dict, combo_request_obj.id, date, agri_officer)

print('insterted')   

run
53 1
54 257
[{'id': 53, 'entered_qty': 1}, {'id': 54, 'entered_qty': 29}]
286
2021-02-25
crossed
crossed
crossed
Parameswaran_sargur   S21F11M2  -  S21F11M2
crossed
crossed
crossed
Parameswaran_sargur   S21F12M2  -  S21F130M2
success
------------------------
run
53 0
54 228
[{'id': 53, 'entered_qty': 0}, {'id': 54, 'entered_qty': 40}]
287
2021-02-25


/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.expected_date received a naive datetime (2021-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.issue_rised_date received a naive datetime (2021-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/kultivate/Softwares/vir

crossed
crossed
crossed
crossed
crossed
crossed
Parameswaran_hd_kote   S21F131M2  -  S21F170M2
success
------------------------
run
run
53 0
54 188
[{'id': 53, 'entered_qty': 0}, {'id': 54, 'entered_qty': 20}]
288
2021-02-25
crossed
crossed
crossed
crossed
crossed
crossed


/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:57:06.915028) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:57:06.941850) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:57:06.944471) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s

Puttamallappa   S21F171M2  -  S21F190M2
success
------------------------
run
53 0
54 168
[{'id': 53, 'entered_qty': 0}, {'id': 54, 'entered_qty': 20}]
289
2021-02-25
crossed
crossed
crossed
crossed
crossed
crossed
Mahadeva Prasad   S21F191M2  -  S21F1110M2
success
------------------------
run
53 0
54 148
[{'id': 53, 'entered_qty': 0}, {'id': 54, 'entered_qty': 20}]
290
2021-02-25
crossed
crossed
crossed
crossed
crossed
crossed


...

/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:57:22.199940) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:57:22.203282) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:57:22.206261) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datet

crossed
crossed
crossed
success
------------------------
run
53 0
[{'id': 53, 'entered_qty': 0}]
404
2021-04-04
crossed
crossed
crossed
success
------------------------
run
53 0
[{'id': 53, 'entered_qty': 0}]
405
2021-07-04
crossed
crossed
crossed
success
------------------------
run
53 0
54 0
55 0
56 0
57 0
58 0
[{'id': 53, 'entered_qty': 0}, {'id': 54, 'entered_qty': 0}, {'id': 55, 'entered_qty': 0}, {'id': 56, 'entered_qty': 0}, {'id': 57, 'entered_qty': 0}, {'id': 58, 'entered_qty': 0}]
406
2021-09-04
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
success
------------------------
run
53 0
[{'id': 53, 'entered_qty': 0}]
407
2021-09-04
crossed
crossed
crossed
success
------------------------
run
53 0
[{'id': 53, 'entered_qty': 0}]
408
2021-09-04
crossed
crossed
crossed
success
------------------------
run
53 0
[{'id': 53, 'entered_qty': 0}]
409
2021-04-14
crossed
crossed
crossed
success
-

/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:57:22.401322) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:57:22.405963) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:57:22.427954) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datet

crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
success
------------------------
run
53 0
54 0
55 0
56 0
57 0
58 0
[{'id': 53, 'entered_qty': 0}, {'id': 54, 'entered_qty': 0}, {'id': 55, 'entered_qty': 0}, {'id': 56, 'entered_qty': 0}, {'id': 57, 'entered_qty': 0}, {'id': 58, 'entered_qty': 0}]
411
2021-04-14
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
crossed
success
------------------------
run
53 0
[{'id': 53, 'entered_qty': 0}]
412
2021-04-15
crossed
crossed
crossed
success
------------------------
run
53 0
[{'id': 53, 'entered_qty': 0}]
413
2021-04-15
crossed
crossed
crossed
success
------------------------
insterted


/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:57:22.606923) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField InputSubStoreInventory.quantity_now_time received a naive datetime (2021-05-03 20:57:22.609081) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/kultivate/Softwares/virtualenv/ccgb/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2021-05-03 20:57:22.613103) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s

In [4]:
import pandas as pd
pd.DataFrame(list(InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo__id__in=[9]).order_by('id').values()))

,id,season_id,input_store_inventory_id,sub_storage_id,section,sub_section,date_of_receipt,quantity_at_receipt,quantity_now,quantity_now_time,unit_id,label_range_start,label_range_end,created_by_id,modified_by_id,time_modified,time_created
0,53,3,64,1,,,2021-05-03,1,1,2021-05-03 20:50:41.175319+00:00,4,S21F11M2,S21F11M2,116,116,2021-05-03 20:50:41.185138+00:00,2021-05-03 20:50:41.175978+00:00
1,54,3,65,1,,,2021-05-03,257,257,2021-05-03 20:50:41.199399+00:00,4,S21F12M2,S21F1258M2,116,116,2021-05-03 20:50:42.837741+00:00,2021-05-03 20:50:41.199827+00:00
2,55,3,66,1,,,2021-05-03,516,516,2021-05-03 20:50:42.845641+00:00,4,S21F1259M2,S21F1774M2,116,116,2021-05-03 20:50:45.560859+00:00,2021-05-03 20:50:42.845965+00:00
3,56,3,67,1,,,2021-05-03,257,257,2021-05-03 20:50:45.568342+00:00,4,S21F1775M2,S21F11031M2,116,116,2021-05-03 20:50:46.898123+00:00,2021-05-03 20:50:45.569017+00:00
4,57,3,68,1,,,2021-05-03,1031,1031,2021-05-03 20:50:46.905622+00:00,4,S21F11032M2,S21F12062M2,116,116,2021-05-03 20:50:52.595061+00:00,2021-05-03 20:50:46.906122+00:00
5,58,3,69,1,,,2021-05-03,448,448,2021-05-03 20:50:52.601593+00:00,4,S21F12063M2,S21F12510M2,116,116,2021-05-03 20:50:55.167474+00:00,2021-05-03 20:50:52.602036+00:00
